# List of Model Available 

In [2]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 13.4 MB/s 
     |████████████████████████████████| 182 kB 97.3 MB/s 


In [3]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
print(model_names)

['adv_inception_v3', 'bat_resnext26ts', 'beit_base_patch16_224', 'beit_base_patch16_224_in22k', 'beit_base_patch16_384', 'beit_large_patch16_224', 'beit_large_patch16_224_in22k', 'beit_large_patch16_384', 'beit_large_patch16_512', 'beitv2_base_patch16_224', 'beitv2_base_patch16_224_in22k', 'beitv2_large_patch16_224', 'beitv2_large_patch16_224_in22k', 'botnet26t_256', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384', 'cait_s36_384', 'cait_xs24_384', 'cait_xxs24_224', 'cait_xxs24_384', 'cait_xxs36_224', 'cait_xxs36_384', 'coat_lite_mini', 'coat_lite_small', 'coat_lite_tiny', 'coat_mini', 'coat_tiny', 'coatnet_0_rw_224', 'coatnet_1_rw_224', 'coatnet_bn_0_rw_224', 'coatnet_nano_rw_224', 'coatnet_rmlp_1_rw_224', 'coatnet_rmlp_2_rw_224', 'coatnet_rmlp_nano_rw_224', 'coatnext_nano_rw_224', 'convit_base', 'convit_small', 'convit_tiny', 'convmixer_768_32', 'convmixer_1024_20_ks9_p14', 'convmixer_1536_20', 'convnext_atto', 'convnext_atto_ols', 'convnext_base', 'convnext_base_384_in

In [ ]:
#import timm
#model = timm.create_model('adv_inception_v3', pretrained=True)
#model.eval()


## Imagenet - sketch Data

In [4]:
import numpy as np
import sklearn.metrics as sk
recall_level_default = 0.95

def calib_err(confidence, correct, p='2', beta=100):
    # beta is target bin size
    idxs = np.argsort(confidence)
    confidence = confidence[idxs]
    correct = correct[idxs]
    bins = [[i * beta, (i + 1) * beta] for i in range(len(confidence) // beta)]
    bins[-1] = [bins[-1][0], len(confidence)]

    cerr = 0
    total_examples = len(confidence)
    for i in range(len(bins) - 1):
        bin_confidence = confidence[bins[i][0]:bins[i][1]]
        bin_correct = correct[bins[i][0]:bins[i][1]]
        num_examples_in_bin = len(bin_confidence)

        if num_examples_in_bin > 0:
            difference = np.abs(np.nanmean(bin_confidence) - np.nanmean(bin_correct))

            if p == '2':
                cerr += num_examples_in_bin / total_examples * np.square(difference)
            elif p == '1':
                cerr += num_examples_in_bin / total_examples * difference
            elif p == 'infty' or p == 'infinity' or p == 'max':
                cerr = np.maximum(cerr, difference)
            else:
                assert False, "p must be '1', '2', or 'infty'"

    if p == '2':
        cerr = np.sqrt(cerr)

    return cerr


def aurra(confidence, correct):
    conf_ranks = np.argsort(confidence)[::-1]  # indices from greatest to least confidence
    rra_curve = np.cumsum(np.asarray(correct)[conf_ranks])
    rra_curve = rra_curve / np.arange(1, len(rra_curve) + 1)  # accuracy at each response rate
    return np.mean(rra_curve)


def soft_f1(confidence, correct):
    wrong = 1 - correct

    # # the incorrectly classified samples are our interest
    # # so they make the positive class
    # tp_soft = np.sum((1 - confidence) * wrong)
    # fp_soft = np.sum((1 - confidence) * correct)
    # fn_soft = np.sum(confidence * wrong)

    # return 2 * tp_soft / (2 * tp_soft + fn_soft + fp_soft)
    return 2 * ((1 - confidence) * wrong).sum()/(1 - confidence + wrong).sum()


def tune_temp(logits, labels, binary_search=True, lower=0.2, upper=5.0, eps=0.0001):
    logits = np.array(logits)

    if binary_search:
        import torch
        import torch.nn.functional as F

        logits = torch.FloatTensor(logits)
        labels = torch.LongTensor(labels)
        t_guess = torch.FloatTensor([0.5*(lower + upper)]).requires_grad_()

        while upper - lower > eps:
            if torch.autograd.grad(F.cross_entropy(logits / t_guess, labels), t_guess)[0] > 0:
                upper = 0.5 * (lower + upper)
            else:
                lower = 0.5 * (lower + upper)
            t_guess = t_guess * 0 + 0.5 * (lower + upper)

        t = min([lower, 0.5 * (lower + upper), upper], key=lambda x: float(F.cross_entropy(logits / x, labels)))
    else:
        import cvxpy as cx

        set_size = np.array(logits).shape[0]

        t = cx.Variable()

        expr = sum((cx.Minimize(cx.log_sum_exp(logits[i, :] * t) - logits[i, labels[i]] * t)
                    for i in range(set_size)))
        p = cx.Problem(expr, [lower <= t, t <= upper])

        p.solve()   # p.solve(solver=cx.SCS)
        t = 1 / t.value

    return t


def print_measures(rms, aurra_metric, mad, sf1, method_name='Baseline'):
    print('\t\t\t\t\t\t\t' + method_name)
    print('RMS Calib Error (%): \t\t{:.2f}'.format(100 * rms))
    print('AURRA (%): \t\t\t{:.2f}'.format(100 * aurra))
    # print('MAD Calib Error (%): \t\t{:.2f}'.format(100 * mad))
    # print('Soft F1 Score (%):   \t\t{:.2f}'.format(100 * sf1))


def show_calibration_results(confidence, correct, method_name='Baseline'):

    print('\t\t\t\t' + method_name)
    print('RMS Calib Error (%): \t\t{:.2f}'.format(
        100 * calib_err(confidence, correct, p='2')))

    print('AURRA (%): \t\t\t{:.2f}'.format(
        100 * aurra(confidence, correct)))

    # print('MAD Calib Error (%): \t\t{:.2f}'.format(
    #     100 * calib_err(confidence, correct, p='1')))

    # print('Soft F1-Score (%): \t\t{:.2f}'.format(
    #     100 * soft_f1(confidence, correct)))

def fpr_and_fdr_at_recall(y_true, y_score, recall_level=recall_level_default, pos_label=None):
    classes = np.unique(y_true)
    if (pos_label is None and
            not (np.array_equal(classes, [0, 1]) or
                     np.array_equal(classes, [-1, 1]) or
                     np.array_equal(classes, [0]) or
                     np.array_equal(classes, [-1]) or
                     np.array_equal(classes, [1]))):
        raise ValueError("Data is not binary and pos_label is not specified")
    elif pos_label is None:
        pos_label = 1.

    # make y_true a boolean vector
    y_true = (y_true == pos_label)

    # sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]

    # y_score typically has many tied values. Here we extract
    # the indices associated with the distinct values. We also
    # concatenate a value for the end of the curve.
    distinct_value_indices = np.where(np.diff(y_score))[0]
    threshold_idxs = np.r_[distinct_value_indices, y_true.size - 1]

    # accumulate the true positives with decreasing threshold
    tps = stable_cumsum(y_true)[threshold_idxs]
    fps = 1 + threshold_idxs - tps      # add one because of zero-based indexing

    thresholds = y_score[threshold_idxs]

    recall = tps / tps[-1]

    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)      # [last_ind::-1]
    recall, fps, tps, thresholds = np.r_[recall[sl], 1], np.r_[fps[sl], 0], np.r_[tps[sl], 0], thresholds[sl]

    cutoff = np.argmin(np.abs(recall - recall_level))

    return fps[cutoff] / (np.sum(np.logical_not(y_true)))   # , fps[cutoff]/(fps[cutoff] + tps[cutoff])

def get_measures(_pos, _neg, recall_level=recall_level_default):
    pos = np.array(_pos[:]).reshape((-1, 1))
    neg = np.array(_neg[:]).reshape((-1, 1))
    examples = np.squeeze(np.vstack((pos, neg)))
    labels = np.zeros(len(examples), dtype=np.int32)
    labels[:len(pos)] += 1

    auroc = sk.roc_auc_score(labels, examples)
    aupr = sk.average_precision_score(labels, examples)
    fpr = fpr_and_fdr_at_recall(labels, examples, recall_level)

    return auroc, aupr, fpr


def print_measures_old(auroc, aupr, fpr, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}'.format(int(100 * recall_level), 100 * fpr))
    print('AUROC: \t{:.2f}'.format(100 * auroc))
    print('AUPR:  \t{:.2f}'.format(100 * aupr))


def print_measures_with_std(aurocs, auprs, fprs, method_name='Ours', recall_level=recall_level_default):
    print('\t\t\t' + method_name)
    print('FPR{:d}:\t{:.2f}\t+/- {:.2f}'.format(int(100 * recall_level), 100 * np.mean(fprs), 100 * np.std(fprs)))
    print('AUROC: \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(aurocs), 100 * np.std(aurocs)))
    print('AUPR:  \t{:.2f}\t+/- {:.2f}'.format(100 * np.mean(auprs), 100 * np.std(auprs)))


def get_and_print_results(out_score, in_score, num_to_avg=1):

    aurocs, auprs, fprs = [], [], []
    #for _ in range(num_to_avg):
    #    out_score = get_ood_scores(ood_loader)
    measures = get_measures(out_score, in_score)
    aurocs.append(measures[0]); auprs.append(measures[1]); fprs.append(measures[2])

    auroc = np.mean(aurocs); aupr = np.mean(auprs); fpr = np.mean(fprs)
    #auroc_list.append(auroc); aupr_list.append(aupr); fpr_list.append(fpr)

    #if num_to_avg >= 5:
    #    print_measures_with_std(aurocs, auprs, fprs, method_name='Ours')
    #else:
    #    print_measures(auroc, aupr, fpr, method_name='Ours')
    return auroc, aupr, fpr 

# Imagenet - sketch: Dataloader

In [5]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as trn
import torchvision.transforms.functional as trnF
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import numpy as np

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

test_transform = trn.Compose(
    [trn.Resize(256), trn.CenterCrop(224), trn.ToTensor(), trn.Normalize(mean, std)])


PATH_TO_IMAGENET_SKETCH = "/content/drive/MyDrive/vision_project/sketch"

imagenet_sketch = dset.ImageFolder(root=PATH_TO_IMAGENET_SKETCH, transform=test_transform)
nae_loader_sketch = torch.utils.data.DataLoader(imagenet_sketch, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [6]:
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.to('cpu').numpy()
def get_predictions_sketch(loader, net=None, num_cls=1000):
    net = timm.create_model(net, pretrained=True, num_classes=num_cls)
    net.cuda()
    net.eval()
    confidence = []
    correct = []
    num_correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.cuda(), target.cuda()
            output = net(data)

            # accuracy
            pred = output.data.max(1)[1]
            num_correct += pred.eq(target.data).sum().item()

            confidence.extend(to_np(F.softmax(output, dim=1).max(1)[0]).squeeze().tolist())
            pred = output.data.max(1)[1]
            correct.extend(pred.eq(target).to('cpu').numpy().squeeze().tolist())

    return np.array(confidence), np.array(correct), num_correct

def get_imagenet_sketch_results(loader, net=None, num_cls=1000):
    confidence, correct, num_correct = get_predictions_sketch(loader, net, num_cls)
    acc = num_correct / len(loader.dataset)
    print('Accuracy (%):', round(100*acc, 2))
    #show_calibration_results(confidence, correct)
    return acc, confidence.copy(), correct.copy()

# Example

# Recipe - 1 Running - CNN Model

In [ ]:
cnn_model_lst1 = ['cspresnet50', 'densenet121', 'dla34', 'dpn68', 'efficientnetv2_rw_m',
                  'efficientnet_b0', 'mixnet_l', 'mnasnet_100', 'fbnetc_100', 'spnasnet_100', 'tinynet_a',
                 'ghostnet_100', 'hrnet_w18', 'inception_resnet_v2', 'inception_v3', 'nasnetalarge', 'nf_resnet50', 'pnasnet5large',
                 'res2net50_14w_8s', 'resnest14d', 'resnet10t', 'rexnet_100', 'selecsls42b', 'skresnet18',
                  'tnt_s_patch16_224', 'vgg19', 'ese_vovnet19b_dw', 'xception']

In [ ]:
cnn_model_lst1 = ['selecsls42b', 'skresnet18',
                  'tnt_s_patch16_224', 'vgg19', 'ese_vovnet19b_dw', 'xception']

In [ ]:
len(cnn_model_lst1)

6

In [ ]:
cnn_accuracy = []
for name in cnn_model_lst1:
  acc, test_confidence, test_correct = get_imagenet_sketch_results(loader = nae_loader_sketch,net= name)
  value = f'ImageNet-Sketch Accuracy (%) for {name} model: {round(100*acc, 4)}'
  cnn_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-selecsls/selecsls42b-8af30141.pth" to /root/.cache/torch/hub/checkpoints/selecsls42b-8af30141.pth


Accuracy (%): 23.98


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/skresnet18_ra-4eec2804.pth" to /root/.cache/torch/hub/checkpoints/skresnet18_ra-4eec2804.pth


Accuracy (%): 24.88


Downloading: "https://github.com/contrastive/pytorch-image-models/releases/download/TNT/tnt_s_patch16_224.pth.tar" to /root/.cache/torch/hub/checkpoints/tnt_s_patch16_224.pth.tar


...

Accuracy (%): 17.94


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/ese_vovnet19b_dw-a8741004.pth" to /root/.cache/torch/hub/checkpoints/ese_vovnet19b_dw-a8741004.pth


Accuracy (%): 23.84


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


Accuracy (%): 18.26


In [ ]:
cnn_accuracy

['ImageNet-Sketch Accuracy (%) for selecsls42b model: 23.9777',
 'ImageNet-Sketch Accuracy (%) for skresnet18 model: 24.8816',
 'ImageNet-Sketch Accuracy (%) for tnt_s_patch16_224 model: 21.0281',
 'ImageNet-Sketch Accuracy (%) for vgg19 model: 17.9371',
 'ImageNet-Sketch Accuracy (%) for ese_vovnet19b_dw model: 23.8441',
 'ImageNet-Sketch Accuracy (%) for xception model: 18.2554']

In [ ]:
## ignor
#cnn_model_lst2 = ['efficientnet_b0', 'mixnet_l', 'mnasnet_100', 'fbnetc_100', 'spnasnet_100', 'tinynet_a',
#                 'ghostnet_100', 'hrnet_w18', 'inception_resnet_v2', 'nasnetalarge', 'pnasnet5large',
#                 'res2net50_14w_8s', 'resnest14d', 'resnet10t', 'rexnet_100', 'selecsls42b', 'skresnet18',
#                  'tnt_s_patch16_224', 'vgg19', 'ese_vovnet19b_dw', 'xception']

In [ ]:
#cnn_accuracy2 = []

#for name in cnn_model_lst2:
#  acc, test_confidence, test_correct = get_net_results(model_name= name)
#  value = f'ImageNet-A Accuracy (%) for {name} model: {round(100*acc, 4)}'
#  cnn_accuracy2.append(value)
#  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

In [ ]:
#cnn_accuracy2

# Recipe - 2 - Running - Vision Transformer Model

In [ ]:
## vit_large_patch16_384 = An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale

In [ ]:
## divide the model by 4-5 
vit_model_lst = ['beit_base_patch16_224', 'cait_s24_224', 'coatnet_0_rw_224', 'crossvit_base_240', 
                 'deit_base_patch16_224', 'deit3_base_patch16_224', 'efficientformer_l1', 'gcvit_base',
                 'maxvit_tiny_rw_224', 'mobilevit_s', 'mvitv2_base', 'jx_nest_base', 'pit_b_224', 'pvt_v2_b0', 'swin_s3_base_224', 'swinv2_cr_small_224', 'tnt_s_patch16_224', 'twins_pcpvt_base',
                 'visformer_small', 'vit_base_patch32_224', 'volo_d1_224', 'xcit_small_12_p8_224']

In [7]:
vit_model_lst = ['jx_nest_base', 
                 'pit_b_224', 'pvt_v2_b0', 'swin_s3_base_224', 'swinv2_cr_small_224', 'tnt_s_patch16_224', 'twins_pcpvt_base',
                 'visformer_small', 'vit_base_patch32_224', 'volo_d1_224', 'xcit_small_12_p8_224']

In [8]:
vit_accuracy = []

for name in vit_model_lst:
  acc, test_confidence, test_correct = get_imagenet_sketch_results(loader = nae_loader_sketch,net= name)
  value = f'ImageNet-Sketch Accuracy (%) for {name} model: {round(100*acc, 4)}'
  vit_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vt3p-weights/jx_nest_base-8bc41011.pth" to /root/.cache/torch/hub/checkpoints/jx_nest_base-8bc41011.pth


Accuracy (%): 33.36


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-pit-weights/pit_b_820.pth" to /root/.cache/torch/hub/checkpoints/pit_b_820.pth


Accuracy (%): 32.7


Downloading: "https://github.com/whai362/PVT/releases/download/v2/pvt_v2_b0.pth" to /root/.cache/torch/hub/checkpoints/pvt_v2_b0.pth


...

Accuracy (%): 30.11
Accuracy (%): 17.36


Downloading: "https://github.com/sail-sg/volo/releases/download/volo_1/d1_224_84.2.pth.tar" to /root/.cache/torch/hub/checkpoints/d1_224_84.2.pth.tar


Accuracy (%): 36.11


Downloading: "https://dl.fbaipublicfiles.com/xcit/xcit_small_12_p8_224.pth" to /root/.cache/torch/hub/checkpoints/xcit_small_12_p8_224.pth


Accuracy (%): 34.53


In [9]:
vit_accuracy

['ImageNet-Sketch Accuracy (%) for jx_nest_base model: 33.3569',
 'ImageNet-Sketch Accuracy (%) for pit_b_224 model: 32.6986',
 'ImageNet-Sketch Accuracy (%) for pvt_v2_b0 model: 21.7238',
 'ImageNet-Sketch Accuracy (%) for swin_s3_base_224 model: 34.367',
 'ImageNet-Sketch Accuracy (%) for swinv2_cr_small_224 model: 30.991',
 'ImageNet-Sketch Accuracy (%) for tnt_s_patch16_224 model: 21.0281',
 'ImageNet-Sketch Accuracy (%) for twins_pcpvt_base model: 32.5729',
 'ImageNet-Sketch Accuracy (%) for visformer_small model: 30.1126',
 'ImageNet-Sketch Accuracy (%) for vit_base_patch32_224 model: 17.3633',
 'ImageNet-Sketch Accuracy (%) for volo_d1_224 model: 36.108',
 'ImageNet-Sketch Accuracy (%) for xcit_small_12_p8_224 model: 34.532']

# Recipe - 3 - Combination of CNN and Vision Transformer

In [10]:
combine_model_lst = ['convnext_base', 'convit_base', 'edgenext_base', 'levit_128']

In [11]:
combine_accuracy = []

for name in combine_model_lst:
  acc, test_confidence, test_correct = get_imagenet_sketch_results(loader = nae_loader_sketch,net= name)
  value = f'ImageNet-Sketch Accuracy (%) for {name} model: {round(100*acc, 4)}'
  combine_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_1k_224_ema.pth


Accuracy (%): 38.22


Downloading: "https://dl.fbaipublicfiles.com/convit/convit_base.pth" to /root/.cache/torch/hub/checkpoints/convit_base.pth


Accuracy (%): 35.53


Downloading: "https://github.com/mmaaz60/EdgeNeXt/releases/download/v1.2/edgenext_base_usi.pth" to /root/.cache/torch/hub/checkpoints/edgenext_base_usi.pth


Accuracy (%): 37.13


Downloading: "https://dl.fbaipublicfiles.com/LeViT/LeViT-128-b88c2750.pth" to /root/.cache/torch/hub/checkpoints/LeViT-128-b88c2750.pth


Accuracy (%): 26.39


In [12]:
combine_accuracy

['ImageNet-Sketch Accuracy (%) for convnext_base model: 38.2244',
 'ImageNet-Sketch Accuracy (%) for convit_base model: 35.5303',
 'ImageNet-Sketch Accuracy (%) for edgenext_base model: 37.1298',
 'ImageNet-Sketch Accuracy (%) for levit_128 model: 26.3927']

# Recipe - 4 - Running MLP Mixer

In [13]:
mlp_model_lst = ['mixer_b16_224']

In [14]:
mlp_accuracy = []

for name in mlp_model_lst:
  acc, test_confidence, test_correct = get_imagenet_sketch_results(loader = nae_loader_sketch,net= name)
  value = f'ImageNet-Sketch Accuracy (%) for {name} model: {round(100*acc, 4)}'
  mlp_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_mixer_b16_224-76587d61.pth" to /root/.cache/torch/hub/checkpoints/jx_mixer_b16_224-76587d61.pth


Accuracy (%): 19.32


In [15]:
mlp_accuracy

['ImageNet-Sketch Accuracy (%) for mixer_b16_224 model: 19.3224']

# 5.    Recipe - 5 - Mobilenet

In [16]:
mobilenet_model_lst = ['mobilenetv3_large_100']

In [17]:
mobilenet_accuracy = []

for name in mobilenet_model_lst:
  acc, test_confidence, test_correct = get_imagenet_sketch_results(loader = nae_loader_sketch,net= name)
  value = f'ImageNet-Sketch Accuracy (%) for {name} model: {round(100*acc, 4)}'
  mobilenet_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /root/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


Accuracy (%): 22.95


In [18]:
mobilenet_accuracy

['ImageNet-Sketch Accuracy (%) for mobilenetv3_large_100 model: 22.9499']

# 6.  Recipe - 6 - Others

In [19]:
others_model_lst = ['regnetv_040', 'sequencer2d_l']

In [20]:
others_accuracy = []

for name in others_model_lst:
  acc, test_confidence, test_correct = get_imagenet_sketch_results(loader = nae_loader_sketch,net= name)
  value = f'ImageNet-Sketch Accuracy (%) for {name} model: {round(100*acc, 4)}'
  others_accuracy.append(value)
  #show_calibration_results(np.array(test_confidence), np.array(test_correct))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tpu-weights/regnetv_040_ra3-c248f51f.pth" to /root/.cache/torch/hub/checkpoints/regnetv_040_ra3-c248f51f.pth


Accuracy (%): 32.93


Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth


Accuracy (%): 35.87


In [21]:
others_accuracy

['ImageNet-Sketch Accuracy (%) for regnetv_040 model: 32.9285',
 'ImageNet-Sketch Accuracy (%) for sequencer2d_l model: 35.8683']

# 7.  Recipe - 7 - Perceiver

# Cross Checking Model Name

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

In [ ]:
model = timm.create_model('dpn92')
model